### Creating the Model with Tensorflow

In [105]:
import pandas as pd
import tensorflow as tf

In [106]:
df_preprocessed = pd.read_csv('Absenteeism_preprocessed.csv')
df_preprocessed.describe()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
count,700.000000,700.000000,700.000000,700.000000,700.00000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000
mean,0.250000,0.008571,0.090000,0.945714,6.36000,2.011429,222.347143,29.892857,36.417143,271.801774,26.737143,0.167143,1.021429,0.687143,6.761429
std,0.433322,0.092250,0.286386,0.226743,3.50501,1.480396,66.312960,14.804446,6.379083,40.021804,4.254701,0.373370,1.112215,1.166095,12.670082
min,0.000000,0.000000,0.000000,0.000000,1.00000,0.000000,118.000000,5.000000,27.000000,205.917000,19.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,1.000000,3.00000,1.000000,179.000000,16.000000,31.000000,241.476000,24.000000,0.000000,0.000000,0.000000,2.000000
50%,0.000000,0.000000,0.000000,1.000000,6.00000,2.000000,225.000000,26.000000,37.000000,264.249000,25.000000,0.000000,1.000000,0.000000,3.000000
75%,0.250000,0.000000,0.000000,1.000000,10.00000,3.000000,260.000000,50.000000,40.000000,294.217000,31.000000,0.000000,2.000000,1.000000,8.000000
max,1.000000,1.000000,1.000000,1.000000,12.00000,6.000000,388.000000,52.000000,58.000000,378.884000,38.000000,1.000000,4.000000,8.000000,120.000000


#### Target data for classification
* set median value as the threshold to cut samples into excessive absence and regular absence
* using median, the resulting dataset will be balanced between the two classes
* But, keep in mind that such decision should be made according to the requests and questions you are trying to answer.

In [107]:
med = df_preprocessed['Absenteeism Time in Hours'].median(axis=0)
med

3.0

In [108]:
targets = df_preprocessed['Absenteeism Time in Hours'].map(lambda x: 1 if (x>med) else 0)
df_preprocessed['Excessively Absenteeism'] = targets
df_with_targets = df_preprocessed.drop(['Absenteeism Time in Hours'], axis=1)

In [109]:
df_with_targets.describe()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Excessively Absenteeism
count,700.000000,700.000000,700.000000,700.000000,700.00000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000
mean,0.250000,0.008571,0.090000,0.945714,6.36000,2.011429,222.347143,29.892857,36.417143,271.801774,26.737143,0.167143,1.021429,0.687143,0.455714
std,0.433322,0.092250,0.286386,0.226743,3.50501,1.480396,66.312960,14.804446,6.379083,40.021804,4.254701,0.373370,1.112215,1.166095,0.498391
min,0.000000,0.000000,0.000000,0.000000,1.00000,0.000000,118.000000,5.000000,27.000000,205.917000,19.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,1.000000,3.00000,1.000000,179.000000,16.000000,31.000000,241.476000,24.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,1.000000,6.00000,2.000000,225.000000,26.000000,37.000000,264.249000,25.000000,0.000000,1.000000,0.000000,0.000000
75%,0.250000,0.000000,0.000000,1.000000,10.00000,3.000000,260.000000,50.000000,40.000000,294.217000,31.000000,0.000000,2.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,12.00000,6.000000,388.000000,52.000000,58.000000,378.884000,38.000000,1.000000,4.000000,8.000000,1.000000


#### select the inputs

In [110]:
unscaled_inputs = df_with_targets.iloc[:, :-1]

In [111]:
dummy_cols = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4']

# tobe_scaled_cols = ['Month Value',
#        'Day of the Week', 'Transportation Expense', 'Distance to Work',
#        'Age', 'Daily Work Load Average', 'Body Mass Index', 'Education',
#        'Children', 'Pets']
tobe_scaled_cols = unscaled_inputs.loc[:, ~unscaled_inputs.columns.isin(dummy_cols)].columns.values

In [112]:
from sklearn.preprocessing import StandardScaler
absenteeism_scaler = StandardScaler()
absenteeism_scaler.fit(unscaled_inputs[tobe_scaled_cols])
scaled_inputs = absenteeism_scaler.transform(unscaled_inputs[tobe_scaled_cols])
scaled_inputs_df = pd.DataFrame(data=scaled_inputs, columns=tobe_scaled_cols)
scaled_inputs_df = pd.concat([unscaled_inputs[dummy_cols], scaled_inputs_df], axis=1)
scaled_inputs_df.describe()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets
count,700.000000,700.000000,700.000000,700.000000,7.000000e+02,7.000000e+02,7.000000e+02,7.000000e+02,7.000000e+02,7.000000e+02,7.000000e+02,700.000000,7.000000e+02,7.000000e+02
mean,0.250000,0.008571,0.090000,0.945714,-8.120488e-17,-1.040438e-16,-2.131628e-16,5.582836e-17,1.319579e-16,-8.526513e-16,1.446462e-16,0.000000,9.135549e-17,-1.268826e-17
std,0.433322,0.092250,0.286386,0.226743,1.000715e+00,1.000715e+00,1.000715e+00,1.000715e+00,1.000715e+00,1.000715e+00,1.000715e+00,1.000715,1.000715e+00,1.000715e+00
min,0.000000,0.000000,0.000000,0.000000,-1.530333e+00,-1.359682e+00,-1.574681e+00,-1.682647e+00,-1.477309e+00,-1.647399e+00,-1.819793e+00,-0.447980,-9.190300e-01,-5.896898e-01
25%,0.000000,0.000000,0.000000,1.000000,-9.593133e-01,-6.837035e-01,-6.541427e-01,-9.390957e-01,-8.498113e-01,-7.582731e-01,-6.437820e-01,-0.447980,-9.190300e-01,-5.896898e-01
50%,0.000000,0.000000,0.000000,1.000000,-1.027836e-01,-7.725463e-03,4.003371e-02,-2.631399e-01,9.143539e-02,-1.888514e-01,-4.085798e-01,-0.447980,-1.928035e-02,-5.896898e-01
75%,0.250000,0.000000,0.000000,1.000000,1.039256e+00,6.682526e-01,5.682114e-01,1.359154e+00,5.620587e-01,5.604758e-01,1.002633e+00,-0.447980,8.804693e-01,2.684866e-01
max,1.000000,1.000000,1.000000,1.000000,1.610276e+00,2.696187e+00,2.499833e+00,1.494345e+00,3.385799e+00,2.677510e+00,2.649049e+00,2.232242,2.679969e+00,6.275721e+00


In [113]:
scaled_inputs_df.shape

(700, 14)

#### Divide the data into train, validation and test
* Instead of manual shuffle and manual division, this time we use sklearn ready function

In [114]:
from sklearn.model_selection import train_test_split
x_train, test_input, y_train, test_target = train_test_split(scaled_inputs_df, targets, train_size=0.9, random_state=42)

#### divide the training data again, this time to extract the validation dataset

In [115]:
# test_input, validation_input, test_target, validation_target = train_test_split(test_input, test_target, train_size=0.5, random_state=42)

In [116]:
tf.reshape(x_train, [-1])
print(x_train.shape)
print(y_train.shape)
# print(validation_input.shape)
# print(validation_target.shape)
print(test_input.shape)
print(test_target.shape)

(630, 14)
(630,)
(70, 14)
(70,)


#### set the Hyperparameters

In [117]:
HIDDEN_LAYER_SIZE = 14
DROPOUT_RATE = 0.5
EPOCHS = 100 # I will set early-stopping function
BATCH_SIZE = 16
OUTPUT_SIZE = 1

In [118]:
initi=tf.keras.initializers.variance_scaling()
model = tf.keras.Sequential([
                            tf.keras.layers.Dense(HIDDEN_LAYER_SIZE, activation='relu'),
                            # tf.keras.layers.Dense(HIDDEN_LAYER_SIZE, activation='relu'),
                            tf.keras.layers.Dense(OUTPUT_SIZE, activation='sigmoid')
                            ])

#### Optimization and loss function

In [119]:
custom_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
# custom_loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# custom_loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

model.compile(optimizer=custom_optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [120]:
# callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5) 

model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1, 
          # validation_data=(validation_input, validation_target), 
          callbacks=[callback])

Epoch 1/100
40/40 [==============================] - 0s 3ms/step - loss: 0.8086 - accuracy: 0.4667
Epoch 2/100
40/40 [==============================] - 0s 3ms/step - loss: 0.7563 - accuracy: 0.4937
Epoch 3/100
40/40 [==============================] - 0s 3ms/step - loss: 0.7238 - accuracy: 0.5079
Epoch 4/100
40/40 [==============================] - 0s 3ms/step - loss: 0.7008 - accuracy: 0.5206
Epoch 5/100
40/40 [==============================] - 0s 3ms/step - loss: 0.6826 - accuracy: 0.5460
Epoch 6/100
40/40 [==============================] - 0s 3ms/step - loss: 0.6696 - accuracy: 0.5778
Epoch 7/100
40/40 [==============================] - 0s 3ms/step - loss: 0.6570 - accuracy: 0.6032
Epoch 8/100
40/40 [==============================] - 0s 3ms/step - loss: 0.6465 - accuracy: 0.6222
Epoch 9/100
40/40 [==============================] - 0s 3ms/step - loss: 0.6360 - accuracy: 0.6302
Epoch 10/100
40/40 [==============================] - 0s 3ms/step - loss: 0.6268 - accuracy: 0.6508
Epoch 11/